In [1]:
# Librerías generales

import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

In [2]:
import os
import numpy as np
import pickle

# Define la carpeta de datos (ajusta esto según el nombre que reciban los datos en Kaggle)/ no me deja poner el mismo nombre de datasalect
data_folder_name = '../input/datainput'  

# Definir una función para obtener los nombres de archivos en una carpeta
def getFileNames(folder_name):
    file_names = os.listdir(folder_name)
    return file_names

print(getFileNames(data_folder_name))

# Lectura de archivos '.npy'
x_test = np.load(f'{data_folder_name}/x_sub_val.npy')
x_train = np.load(f'{data_folder_name}/x_sub_train_aug.npy')

y_train_coarse = np.load(f'{data_folder_name}/y_sub_train_aug_1.npy')
y_train_fine = np.load(f'{data_folder_name}/y_sub_train_aug_0.npy')

y_test_coarse = np.load(f'{data_folder_name}/y_sub_val_1.npy')
y_test_fine = np.load(f'{data_folder_name}/y_sub_val_0.npy')

# Lectura de archivos '.pck'
with open('/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck', "rb") as f:
    coarse_label_names = pickle.load(f)

with open('/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck', "rb") as f:
    fine_label_names = pickle.load(f)

# Información de las dimensiones de los datos
print('Dimensiones de los datos:')
print(x_test.shape)
print(x_train.shape)
print(y_train_coarse.shape)
print(y_train_fine.shape)

print('Cantidad de clases:')
print(np.shape(coarse_label_names))
print(np.shape(fine_label_names))

['x_sub_val.npy', 'y_sub_train_aug_0.npy', 'y_sub_train_aug_1.npy', 'y_sub_val_1.npy', 'x_sub_train_aug.npy', 'y_sub_val_0.npy']
Dimensiones de los datos:
(10000, 32, 32, 3)
(1040000, 32, 32, 3)
(1040000,)
(1040000,)
Cantidad de clases:
(20,)
(100,)


In [3]:
from keras.optimizers import Adam
from keras.layers import Input, Dense, Dropout, Flatten, LayerNormalization,BatchNormalization, Conv2D, MaxPooling2D
from keras.models import Model
from keras.regularizers import l2

In [4]:
#OTRA ESTRUCTURA
# Entrada y normalización
input_layer = Input(shape=(32, 32, 3), name="matrix_input")

# Estructura de la red
x = BatchNormalization(axis=-1, name="normalization_layer")(input_layer)
flatten_layer = Flatten(name="flattened_input")(x)

# Capas ocultas con mayor capacidad y Dropout
dense_512 = Dense(512, activation='relu', name="dense_512")(flatten_layer)
dropout_512 = Dropout(0.2)(dense_512)
dense_256 = Dense(256, activation='relu', name="dense_256")(dropout_512)
dropout_256 = Dropout(0.2)(dense_256)
dense_128 = Dense(128, activation='relu', name="dense_128")(dropout_256)
dense_64 = Dense(64, activation='relu', name="dense_64")(dense_128)


# Fine-grain prediction branch (100 classes)
fine_output = Dense(100,
                           activation='softmax',
                           name='fine_output')(dense_64)

# Coarse-grain prediction branch (20 classes)
coarse_output = Dense(20,
                             activation='softmax',
                             name='coarse_output')(dense_64)

# Defino el modelo con dos salidas
myModel = Model(
    inputs                  = input_layer,
    #outputs                 = [fine_output, coarse_output]
    outputs                 = [fine_output]
)


# Print model summary
myModel.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ matrix_input (InputLayer)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_layer             │ (None, 32, 32, 3)      │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flattened_input (Flatten)       │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_512 (Dense)               │ (None, 512)            │     1,573,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_256 (Dense)               │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_128 (Dense)               │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fine_output (Dense)             │ (None, 100)            │         6,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,752,368 (6.68 MB)

 Trainable params: 1,752,362 (6.68 MB)

 Non-trainable params: 6 (24.00 B)

In [5]:
# Shuffle de los datos
indexes_train   = np.arange(len(x_train))
indexes_test    = np.arange(len(x_test))

np.random.shuffle(indexes_train)
np.random.shuffle(indexes_test)

# Shuffle de datos de train

x_train_dup         = x_train[indexes_train]
y_train_coarse_dup  = y_train_coarse[indexes_train]
y_train_fine_dup    = y_train_fine[indexes_train]

# Shuffle de datos de test
x_test_dup         = x_test[indexes_test]
y_test_coarse_dup  = y_test_coarse[indexes_test]
y_test_fine_dup    = y_test_fine[indexes_test]

In [6]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard, LambdaCallback

In [18]:
#no probe con patience 10 todavia, recomendada por chat :?
rlrop = ReduceLROnPlateau(
    monitor = "val_accuracy",
    factor = 0.2,
    patience = 10,
    verbose = 1,
    min_lr = 1e-6
)


rlrop2 = ReduceLROnPlateau(
    monitor = "accuracy",
    factor = 0.2,
    patience = 5,
    verbose = 1,
    min_lr = 1e-6
)
#no probe con patience 15 todavia
es = EarlyStopping(
    monitor = "val_accuracy",
    patience = 15,
    verbose = 1,
    restore_best_weights = True
)

es2 = EarlyStopping(
    monitor = "accuracy",
    patience = 10,
    verbose = 1,
    restore_best_weights = True
)



mc = ModelCheckpoint(
    "best_weights.weights.h5",
    monitor = "val_accuracy",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,

)

mc2 = ModelCheckpoint(
    "best_weights.weights.h5",
    monitor = "accuracy",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,

)

tb = TensorBoard(
    log_dir = "logs"
)

In [13]:
myModel.compile(
    optimizer               = Adam(learning_rate=1e-3),
    #loss                    = {'fine_output': 'sparse_categorical_crossentropy', 'coarse_output': 'sparse_categorical_crossentropy'},
    loss                    = {'fine_output': 'sparse_categorical_crossentropy'},
    loss_weights            = None,
    #metrics                 = {'fine_output': 'categorical_accuracy', 'coarse_output': 'categorical_accuracy'},
    metrics                 = {'fine_output': 'accuracy'},
    weighted_metrics        = None,
    run_eagerly             = False,
    steps_per_execution     = 1,
    jit_compile             = "auto",
    auto_scale_loss         = True,
)

In [ ]:
#FIT PRUEBA
history = myModel.fit(
    x                       = x_train_dup,
    y                       = {'fine_output': y_train_fine_dup},
    batch_size              = 256,
    epochs                  = 10,
    verbose                 = "auto",
    callbacks               = [LambdaCallback(on_epoch_end=lambda epoch, logs: print(logs if logs is not None else "No logs available")),rlrop,es,mc,mc2,rlrop2,es2],
    validation_split        = 0.0,
    validation_data         = (x_test_dup, {'fine_output': y_test_fine_dup}),
    shuffle                 = True,
    class_weight            = None,
    sample_weight           = None,
    initial_epoch           = 0,
    steps_per_epoch         = None,
    validation_steps        = None,
    validation_batch_size   = None,
    validation_freq         = 1,
)

Epoch 1/10
2074/4063 ━━━━━━━━━━━━━━━━━━━━ 1:38 49ms/step - accuracy: 0.1948 - loss: 3.3181